To execute the commands and view the output as shown in this webpage, execute the following command from command prompt

jupyter notebook

It should open up a webpage and from there create a new notebook.

Note: Page at https://jupyter.readthedocs.io/en/latest/index.html provides good introductory information about Jupyter notebook and its usage.

In [1]:
# import py_entitymatching package
import py_entitymatching as em
import pandas as pd
import re
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from nltk.stem.lancaster import LancasterStemmer
import random
from sets import Set

/home/karan/anaconda2/envs/npy/lib/python2.7/site-packages/scikit_learn-0.18.1-py2.7-linux-x86_64.egg/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


# Steps
### 1. Read the input tables
### 2. Do blocking
### 3. Save the result

## Read the input tables

In [2]:
# specify filepaths for tables A and B. 
path_A = 'spoj/json/spoj_blocking.csv' # patoh_A is the file path where table_A.csv is stored
path_B = 'codechef/data/codechef_blocking.csv' # path_B is the file path where table_B.csv is stored

In [3]:
# read table A; table A has 'ID' as the key attribute
A = em.read_csv_metadata(path_A)

# read table B; table B has 'ID' as the key attribute
B = em.read_csv_metadata(path_B)
A.rename(columns={"Unnamed: 0":"ID"}, inplace=True)
B.rename(columns={"Unnamed: 0":"ID"}, inplace=True)

No handlers could be found for logger "py_entitymatching.io.parsers"


In [4]:
A["words"] = A["description"].fillna("") + A["input"].fillna("") + A["output"].fillna("")
A = A.drop(["description", "input", "output"], axis=1)

In [5]:
B["words"] = B["description"].fillna("") + B["input"].fillna("") + B["output"].fillna("")
B = B.drop(["description", "input", "output"], axis=1)

In [6]:
# Step 1
def std_words(df):
    letters_only = re.sub("[^a-zA-Z]", " ", df["words"]) # letters only, drop numbers & symbols
    words = letters_only.lower().split() # lower case, split by word                  
    stops = set((nltk.corpus.stopwords.words("english") + 
                ['a','b','c','d','e','f','g','h','i','j','k','l','m','n',
                'o','p','q','r','s','t','u','v','w','x','y','z'])) # general stopwords & letters  
    meaningful_words = [w for w in words if not w in stops] # remove stopwords
    return( " ".join( meaningful_words)) # return re-joined string

# Step 2
A["words"] = A.apply(std_words, axis=1)
B["words"] = B.apply(std_words, axis=1)

em.set_key(A, "ID")
em.set_key(B, "ID")

True

In [88]:
A.apply(lambda x:len(Set(x["words"].split(" "))),axis=1).describe()

count    5166.000000
mean       74.995161
std        40.105063
min         3.000000
25%        46.000000
50%        67.000000
75%        96.000000
max       346.000000
dtype: float64

In [87]:
B.apply(lambda x:len(Set(x["words"].split(" "))),axis=1).describe()

count    5393.000000
mean       52.797515
std        34.960875
min         1.000000
25%        31.000000
50%        52.000000
75%        73.000000
max       313.000000
dtype: float64

## Blocking

In [11]:
ab = em.AttrEquivalenceBlocker()
C0 = ab.block_tables(A, B, 'title', 'title',
                    l_output_attrs=['words', 'title', "solve_rate_normalized"], 
                    r_output_attrs=['words', 'title', "solve_rate_normalized"])

In [52]:
ob = em.OverlapBlocker()
C1 = ob.block_tables(A, B, "words", "words", rem_stop_words=True,
                    l_output_attrs=['words', 'title', "solve_rate_normalized"], 
                    r_output_attrs=['words', 'title', "solve_rate_normalized"],
                    overlap_size = 30, n_jobs=2)

0%                          100%
[##############################] | ETA: 00:00:00
Total time elapsed: 00:00:43


In [53]:
C = em.combine_blocker_outputs_via_union([C0, C1])
C

,_id,ltable_ID,rtable_ID,ltable_words,ltable_title,ltable_solve_rate_normalized,rtable_words,rtable_title,rtable_solve_rate_normalized
0,0,0,5166,program use brute force approach order find answer life universe everything precisely rewrite sm...,TEST,38.716293,help problem please check tutorial input output program use brute force approach order find answ...,TEST,0.100593
1,1,1,3710,important activity acm gsm network mobile phone operator acm must build transmitting stations im...,CMEXPR,-0.071428,secret doors contain interesting word puzzle team archaeologists solve open doors way open doors...,WORDS1,0.025228
2,2,1,4999,important activity acm gsm network mobile phone operator acm must build transmitting stations im...,CMEXPR,-0.071428,loda maelk legendary chefcraft players played many games number fit standard bit integer type to...,BMASTER,0.024878
3,3,1,5000,important activity acm gsm network mobile phone operator acm must build transmitting stations im...,CMEXPR,-0.071428,mad scientist name exor invented notation evaluating complex mathematical expressions notation p...,ZENCALC,0.024878
4,4,1,5335,important activity acm gsm network mobile phone operator acm must build transmitting stations im...,CMEXPR,-0.071428,attention challenge problem longest problem statement ever even longer legendary problem cielhac...,WORDNINJ,0.024884
5,5,3,5335,equidivision square array cells partition cells array exactly sets one contiguous cells two cell...,EQDIV,-0.078162,attention challenge problem longest problem statement ever even longer legendary problem cielhac...,WORDNINJ,0.024884
6,6,4,41,people babylon devoted chance games one popular special kind roulette recently old babylonian ta...,BROUL,-0.094220,earth world know exist anymore changed inevitable evolution chef retains former self peoples pla...,CLOWAY,0.024955
7,7,4,501,people babylon devoted chance games one popular special kind roulette recently old babylonian ta...,BROUL,-0.094220,advancement technology humans designed developed large variety machines work machines work much ...,IOPC14D,0.024857
8,8,4,1415,people babylon devoted chance games one popular special kind roulette recently old babylonian ta...,BROUL,-0.094220,smit dreams king state story goes follows fancies create state cities unidirectional roads initi...,GRETHIEF,0.024852
9,9,4,1418,people babylon devoted chance games one popular special kind roulette recently old babylonian ta...,BROUL,-0.094220,guru prakash sabari srivatsan recently qualified participate world finals international chess to...,SHASGAME,0.024852


In [73]:
def my_function(x, y):
    # x, y will be of type pandas series
    
    # get name attribute
    x_words = x['words'].split(" ")
    y_words = y['words'].split(" ")
    X = set(x_words)
    Y = set(y_words)
    if (len(X) > 50 or len(Y) > 50):
        return True
    if em.overlap_coeff(x_words, y_words) > 0.65:
        return False
    else:
        return True

# create black box blocker
bb = em.BlackBoxBlocker()
# set the blackbox function
bb.set_black_box_function(my_function)

In [77]:
C3 = bb.block_tables(A, B, l_output_attrs=['words', 'title', "solve_rate_normalized"], 
                    r_output_attrs=['words', 'title', "solve_rate_normalized"],n_jobs=2, verbose=True)

0%                          100%
[##############################] | ETA: 00:00:00
Total time elapsed: 00:28:52


In [78]:
C3

,_id,ltable_ID,rtable_ID,ltable_words,ltable_title,ltable_solve_rate_normalized,rtable_words,rtable_title,rtable_solve_rate_normalized
0,0,0,249,program use brute force approach order find answer life universe everything precisely rewrite sm...,TEST,38.716293,problem description input,NUMMAX3,0.024909
1,1,0,2247,program use brute force approach order find answer life universe everything precisely rewrite sm...,TEST,38.716293,output,LOC02,0.024854
2,2,9,2247,successfully solving math homework previous task mirko become bored made list large integers lis...,KOMPICI,0.085136,output,LOC02,0.024854
3,3,40,1614,given sequence numbers cyclic shift positions results following sequence many cyclic shifts sati...,PARSUMS,-0.021908,first line contains number test cases cases follow test case contains two integers first line co...,QUARK3,0.024852
4,4,40,1722,given sequence numbers cyclic shift positions results following sequence many cyclic shifts sati...,PARSUMS,-0.021908,test case print,SIMBIRD,0.024907
5,5,40,2247,given sequence numbers cyclic shift positions results following sequence many cyclic shifts sati...,PARSUMS,-0.021908,output,LOC02,0.024854
6,6,47,1722,today birthday seraph seraph love eat chocolate cake unfortunately seraph much eat chocochip max...,BCAKE,-0.184591,test case print,SIMBIRD,0.024907
7,7,47,2247,today birthday seraph seraph love eat chocolate cake unfortunately seraph much eat chocochip max...,BCAKE,-0.184591,output,LOC02,0.024854
8,8,49,1722,given number output many digits number number divisible count occurences digits number first exa...,PUCMM025,0.854730,test case print,SIMBIRD,0.024907
9,9,49,2247,given number output many digits number number divisible count occurences digits number first exa...,PUCMM025,0.854730,output,LOC02,0.024854


In [79]:
C = em.combine_blocker_outputs_via_union([C0, C1, C3])
C

,_id,ltable_ID,rtable_ID,ltable_words,ltable_title,ltable_solve_rate_normalized,rtable_words,rtable_title,rtable_solve_rate_normalized
0,0,0,249,program use brute force approach order find answer life universe everything precisely rewrite sm...,TEST,38.716293,problem description input,NUMMAX3,0.024909
1,1,0,2247,program use brute force approach order find answer life universe everything precisely rewrite sm...,TEST,38.716293,output,LOC02,0.024854
2,2,0,5166,program use brute force approach order find answer life universe everything precisely rewrite sm...,TEST,38.716293,help problem please check tutorial input output program use brute force approach order find answ...,TEST,0.100593
3,3,1,3710,important activity acm gsm network mobile phone operator acm must build transmitting stations im...,CMEXPR,-0.071428,secret doors contain interesting word puzzle team archaeologists solve open doors way open doors...,WORDS1,0.025228
4,4,1,4999,important activity acm gsm network mobile phone operator acm must build transmitting stations im...,CMEXPR,-0.071428,loda maelk legendary chefcraft players played many games number fit standard bit integer type to...,BMASTER,0.024878
5,5,1,5000,important activity acm gsm network mobile phone operator acm must build transmitting stations im...,CMEXPR,-0.071428,mad scientist name exor invented notation evaluating complex mathematical expressions notation p...,ZENCALC,0.024878
6,6,1,5335,important activity acm gsm network mobile phone operator acm must build transmitting stations im...,CMEXPR,-0.071428,attention challenge problem longest problem statement ever even longer legendary problem cielhac...,WORDNINJ,0.024884
7,7,3,5335,equidivision square array cells partition cells array exactly sets one contiguous cells two cell...,EQDIV,-0.078162,attention challenge problem longest problem statement ever even longer legendary problem cielhac...,WORDNINJ,0.024884
8,8,4,41,people babylon devoted chance games one popular special kind roulette recently old babylonian ta...,BROUL,-0.094220,earth world know exist anymore changed inevitable evolution chef retains former self peoples pla...,CLOWAY,0.024955
9,9,4,501,people babylon devoted chance games one popular special kind roulette recently old babylonian ta...,BROUL,-0.094220,advancement technology humans designed developed large variety machines work machines work much ...,IOPC14D,0.024857


In [80]:
# display C1
dbg = em.debug_blocker(C, A, B, output_size=5, verbose=True)
dbg

,_id,similarity,ltable_ID,rtable_ID,ltable_title,ltable_words,rtable_title,rtable_words
0,0,0.444444,231,2107,LGIC,given sequance natural numbers find th term sequence one natural number one natural number th te...,CDVA1501,given sequence natural numbers find th term ansingle line containing natural number nprint th te...
1,1,0.431373,4775,3874,MAIN72,given array integers want find sum integers expressed sum least one subset given array first lin...,COMB4SUM,special sum numbers defined denotes absolute value first line contains number test cases follow ...
2,2,0.428571,2307,3513,CEQU,let us see following equation ax given three positive integers determine whether exists least on...,GOC203,prof vats nit hamirpur given problem whole class imposed condition give attendance solve within ...
3,3,0.428571,4651,4748,NOVICE21,given three integers find many integers inclusive divisible first line contains number test case...,LCM,given compute sum lcm pairs positive integers integer divides give answer modulo first line cont...
4,4,0.428571,4417,2598,CNTTREE,given tree need count many subtrees diameter first line contains number test cases test cases fo...,TRSUM,let lx denote level node rooted tree lx root otherwise lx ly parent rooted tree need calculate s...


In [84]:
left = A.loc[4775]
right = B.loc[3874]
left["words"]

'given array integers want find sum integers expressed sum least one subset given array first line contains number test case test cases follow first line test case contains test case print answer new line example input output'

In [85]:
right["words"]

'special sum numbers defined denotes absolute value first line contains number test cases follow first line test case contains integer second line test case contains array space separated integers test case output sum explained'

In [86]:
print ob.block_tuples(left, right, "words", "words", overlap_size=30)
print bb.block_tuples(left,right)

True
True


## Save the result to disk

In [97]:
C.loc[:,["_id","ltable_ID","rtable_ID"]]

,_id,ltable_ID,rtable_ID
0,0,0,249
1,1,0,2247
2,2,0,5166
3,3,1,3710
4,4,1,4999
5,5,1,5000
6,6,1,5335
7,7,3,5335
8,8,4,41
9,9,4,501


In [98]:
# export C
em.to_csv_metadata(C.loc[:,["_id","ltable_ID","rtable_ID"]], './candidate_set.csv')

True